In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from textblob import Word
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('text_emotion.csv')

data = data.drop('author', axis=1)



print(data)

         tweet_id   sentiment  \
0      1956967341       empty   
1      1956967666     sadness   
2      1956967696     sadness   
3      1956967789  enthusiasm   
4      1956968416     neutral   
...           ...         ...   
39995  1753918954     neutral   
39996  1753919001        love   
39997  1753919005        love   
39998  1753919043   happiness   
39999  1753919049        love   

                                                 content  
0      @tiffanylue i know  i was listenin to bad habi...  
1      Layin n bed with a headache  ughhhh...waitin o...  
2                    Funeral ceremony...gloomy friday...  
3                   wants to hang out with friends SOON!  
4      @dannycastillo We want to trade with someone w...  
...                                                  ...  
39995                                   @JohnLloydTaylor  
39996                     Happy Mothers Day  All my love  
39997  Happy Mother's Day to all the mommies out ther...  
39998  @niari

In [2]:
data = data.drop('tweet_id', axis=1)

In [3]:
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
pd = data.to_numpy()

In [17]:
L = []
for i in range(0,40000):
    if ((pd[i].any()) not in L):
        L.append(pd[i].any())
        

In [18]:
L

['empty',
 'sadness',
 'enthusiasm',
 'neutral',
 'worry',
 'surprise',
 'love',
 'fun',
 'hate',
 'happiness',
 'boredom',
 'relief',
 'anger']

In [20]:
data
data = data.drop('tweet_id', axis=1)

KeyError: "['tweet_id'] not found in axis"

In [21]:
dat = data.to_numpy()

In [22]:
dat

array([['empty',
        '@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =['],
       ['sadness',
        'Layin n bed with a headache  ughhhh...waitin on your call...'],
       ['sadness', 'Funeral ceremony...gloomy friday...'],
       ...,
       ['love',
        "Happy Mother's Day to all the mommies out there, be you woman or man as long as you're 'momma' to someone this is your day!"],
       ['happiness',
        '@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!!  PEEP OUT MY NEW HIT SINGLES WWW.MYSPACE.COM/IPSOHOT I DEF. WAT U IN THE VIDEO!!'],
       ['love',
        '@mopedronin bullet train from tokyo    the gf and i have been visiting japan since thursday  vacation/sightseeing    gaijin godzilla']],
      dtype=object)

In [23]:
empty = []
sadness = []
enthusiasm = []
neutral = []
worry = []
surprise = []
love = []
fun = []
hate = []
happiness = []
boredom = []
relief = []
anger = []

In [24]:
for i in range(0,40000):
    if (dat[i][0]=="empty"):
        empty.append(dat[i][1])
    elif (dat[i][0]=="sadness"):
        sadness.append(dat[i][1])
    elif (dat[i][0]=="enthusiasm"):
        enthusiasm.append(dat[i][1])
    elif (dat[i][0]=="neutral"):
        neutral.append(dat[i][1])
    elif (dat[i][0]=="worry"):
        worry.append(dat[i][1])
    elif (dat[i][0]=="surprise"):
        surprise.append(dat[i][1])
    elif (dat[i][0]=="love"):
        love.append(dat[i][1])
    elif (dat[i][0]=="fun"):
        fun.append(dat[i][1])
    elif (dat[i][0]=="hate"):
        hate.append(dat[i][1])
    elif (dat[i][0]=="happiness"):
        happiness.append(dat[i][1])
    elif (dat[i][0]=="boredom"):
        boredom.append(dat[i][1])
    elif (dat[i][0]=="relief"):
        relief.append(dat[i][1])
    elif (dat[i][0]=="anger"):
        anger.append(dat[i][1])

In [25]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/yahya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yahya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yahya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [26]:
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples

In [27]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def simplifier_phrase(tokens):
    L = WordNetLemmatizer()
    lemmatized_sentence = []
    for mot, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(L.lemmatize(mot, pos))
    return lemmatized_sentence

In [28]:
import re, string

def nettoie_donnees(tweet_tokens, stop_words = ()):

    Liste = []

    for mot, tag in pos_tag(tweet_tokens):
        mot = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', mot)
        mot = re.sub("(@[A-Za-z0-9_]+)","", mot)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        mot = lemmatizer.lemmatize(mot, pos)

        if len(mot) > 0 and mot not in string.punctuation and mot.lower() not in stop_words:
            Liste.append(mot.lower())
    return Liste

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/yahya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def cherche_mots(liste_nettoye):
    for liste in liste_nettoye:
        for mot in liste:
            yield mot
            
def prepare_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [31]:
from nltk.tokenize import word_tokenize

empty_tok = [word_tokenize(i) for i in empty]
sadness_tok = [word_tokenize(i) for i in sadness]
enthusiasm_tok = [word_tokenize(i) for i in enthusiasm]
neutral_tok = [word_tokenize(i) for i in neutral]
worry_tok = [word_tokenize(i) for i in worry]
surprise_tok = [word_tokenize(i) for i in surprise]
love_tok = [word_tokenize(i) for i in love]
fun_tok = [word_tokenize(i) for i in fun]
hate_tok = [word_tokenize(i) for i in hate]
happiness_tok = [word_tokenize(i) for i in happiness]
boredom_tok = [word_tokenize(i) for i in boredom]
relief_tok = [word_tokenize(i) for i in relief]
anger_tok = [word_tokenize(i) for i in anger]


empty_com = []
sadness_com = []
enthusiasm_com = []
neutral_com = []
worry_com = []
surprise_com = []
love_com = []
fun_com = []
hate_com = []
happiness_com = []
boredom_com = []
relief_com = []
anger_com = []


for i in empty_tok:
    empty_com.append(nettoie_donnees(i, stop_words))
for i in sadness_tok:
    sadness_com.append(nettoie_donnees(i, stop_words))
for i in enthusiasm_tok:
    enthusiasm_com.append(nettoie_donnees(i, stop_words))
for i in neutral_tok:
    neutral_com.append(nettoie_donnees(i, stop_words))
for i in worry_com:
    worry_com.append(nettoie_donnees(i, stop_words))
for i in surprise_tok:
    surprise_com.append(nettoie_donnees(i, stop_words))
for i in love_tok:
    love_com.append(nettoie_donnees(i, stop_words))
for i in fun_tok:
    fun_com.append(nettoie_donnees(i, stop_words))
for i in hate_tok:
    hate_com.append(nettoie_donnees(i, stop_words))
for i in happiness_tok:
    happiness_com.append(nettoie_donnees(i, stop_words))
for i in boredom_tok:
    boredom_com.append(nettoie_donnees(i, stop_words))
for i in relief_tok:
    relief_com.append(nettoie_donnees(i, stop_words))
for i in anger_tok:
    anger_com.append(nettoie_donnees(i, stop_words))
    
    
#j = 0
#for i in Commentaire:
 #   print(donnees[j])
  #  j = j + 1
 #   print(i)
 #   print(classifier.classify(dict([token, True] for token in i)))






In [32]:

empty_for_model = prepare_for_model(empty_com)
sadness_for_model = prepare_for_model(sadness_com)
enthusiasm_for_model = prepare_for_model(enthusiasm_com)
neutral_for_model = prepare_for_model(neutral_com)
worry_for_model = prepare_for_model(worry_com)
surprise_for_model = prepare_for_model(surprise_com)
love_for_model = prepare_for_model(love_com)
fun_for_model = prepare_for_model(fun_com)
hate_for_model = prepare_for_model(hate_com)
happiness_for_model = prepare_for_model(happiness_com)
boredom_for_model = prepare_for_model(boredom_com)
relief_for_model = prepare_for_model(relief_com)
anger_for_model = prepare_for_model(anger_com)


empty_dataset = [(i, "empty")
                     for i in empty_for_model]
sadness_dataset = [(i, "sadness")
                     for i in sadness_for_model]
enthusiasm_dataset = [(i, "enthusiasm")
                     for i in enthusiasm_for_model]
neutral_dataset = [(i, "neutral")
                     for i in neutral_for_model]
worry_dataset = [(i, "worry")
                     for i in worry_for_model]
surprise_dataset = [(i, "surprise")
                     for i in surprise_for_model]
love_dataset = [(i, "love")
                     for i in love_for_model]
fun_dataset = [(i, "fun")
                     for i in fun_for_model]
hate_dataset = [(i, "hate")
                     for i in hate_for_model]
happiness_dataset = [(i, "happiness")
                     for i in happiness_for_model]
boredom_dataset = [(i, "boredom")
                     for i in boredom_for_model]
relief_dataset = [(i, "relief")
                     for i in relief_for_model]
anger_dataset = [(i, "anger")
                     for i in anger_for_model]



In [33]:

empty_dataset

[({'bad': True,
   'early': True,
   'freakin': True,
   'habit': True,
   'know': True,
   'listenin': True,
   'part': True,
   'start': True,
   'tiffanylue': True},
  'empty'),
 ({'break': True,
   'cali': True,
   'credit': True,
   'creyes': True,
   'elem': True,
   'graduate': True,
   'high': True,
   'middle': True,
   'need': True,
   'open': True,
   'remain': True,
   'school': True},
  'empty'),
 ({'djmicdamn': True,
   'fucker': True,
   'hey': True,
   'lil': True,
   'textd': True,
   'yu': True},
  'empty'),
 ({'answer': True,
   'basic': True,
   'french-': True,
   'learn': True,
   'never': True,
   'softtouchme': True,
   'stuff-': True,
   'write': True,
   'you-': True},
  'empty'),
 ({"'m": True, 'bed': True, 'go': True, 'goodnight': True, 'headache': True},
  'empty'),
 ({"'m": True,
   '4': True,
   'almost': True,
   'b': True,
   'bore': True,
   'dymediva23': True,
   'frm': True,
   'n': True,
   'nite': True,
   'oppose': True,
   'sleep': True},
  'empt

In [34]:
import random

dataset = empty_dataset + sadness_dataset + enthusiasm_dataset + neutral_dataset + worry_dataset + surprise_dataset + love_dataset + fun_dataset + hate_dataset + happiness_dataset + boredom_dataset + relief_dataset + anger_dataset

random.shuffle(dataset)

train_data = dataset[:22000]
test_data = dataset[22000:]

In [35]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))



Accuracy is: 0.16612514411487264
Most Informative Features
                    hate = True             hate : happin =    238.2 : 1.0
                     gah = True            anger : neutra =    125.8 : 1.0
                    bore = True           boredo : happin =    116.0 : 1.0
                    fuck = True             hate : neutra =     83.8 : 1.0
                 fifteen = True           boredo : neutra =     83.2 : 1.0
                    self = True           boredo : neutra =     83.2 : 1.0
                    tool = True           boredo : neutra =     83.2 : 1.0
                migraine = True            anger : neutra =     75.5 : 1.0
                 goooood = True            anger : neutra =     75.5 : 1.0
              johncmayer = True            anger : neutra =     75.5 : 1.0
None


In [36]:
len(dataset) * 0.7

22078.699999999997

[({'god': True,
   'high': True,
   'left': True,
   'month': True,
   'ready': True,
   'school': True,
   'summer': True,
   'thank': True},
  'relief'),
 ({'75th': True,
   'forward': True,
   'grandparent': True,
   'italian': True,
   'lookin': True,
   'lovely': True,
   'meal': True,
   'posh': True},
  'love'),
 ({"'s": True,
   '*thumbs': True,
   'blakelewis': True,
   'breakfast': True,
   'happy': True,
   'like': True,
   'morning': True,
   'oh': True,
   'right': True,
   'see': True,
   'up*': True},
  'surprise'),
 ({'18.42': True,
   'amp': True,
   'dinner': True,
   'event': True,
   'fantastic': True,
   'great': True,
   'hot': True,
   'life': True,
   'movie': True,
   'perfect': True,
   'quinto': True,
   'recall': True,
   'shop': True,
   'star': True,
   'study': True,
   'trek': True,
   'wk/end': True,
   'zach': True},
  'surprise'),
 ({"'s": True,
   'amazing': True,
   'chrisjlovell': True,
   'dickiea': True,
   'download': True,
   'literally': True,